In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra|
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
tqdm.pandas()

import plotly.express as px
import plotly.graph_objects as go
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 0. จุดประสงค์ของการทำ Data Analytics
> เพื่อหาสิ่งที่ทำให้ Donald Trump แพ้การเลือกตั้ง โดยวิเคราะห์ข้อมูลจาก 2 main parties (DEM, REP)

# 1. การต้ังคำถาม / การตั้งสมมติฐาน

> คำถาม : ทำไม Donald Trump ถึงแพ้การเลือกตั้งปี 2020
<br>
> สมมติฐาน : 
    - 1. Trump เสีย State ซึ่งเคยเป็นฐานเสียงเมื่อการเลือกตั้งปี 2016
    
> คำถาม : อะไรทำให้คะแนน Vote ภายใน State ที่เคยเป็นฐานเสียงของ Trump เมื่อการเลือกตั้งปี 2016 เปลี่ยน
<br>
> สมมติฐาน : 
    - 1. รูปแบบประชากรเปลี่ยนไป 
        - 1.1. New Voters
        - 1.2. Gender Ratio
        - 1.3. Races
    - 2. เพศของ Candidate จากฝั่ง DEM
        - 2.1. Voters หันมาเลือก DEM เพราะ Candidate เป็นผู้ชาย คาดว่า Voters ผู้ชายจะโหวตให้กับ Candidate ผู้ชาย


# 2. การเก็บข้อมูล/รวบรวมข้อมูลดิบ

- 2.1. ข้อมูลการเลือกตั้งปี 2020
    - 2.1.1. ข้อมูลผลการเลือกตั้งปี 2020
    - 2.1.2. ข้อมูล Electrical Votes ปี 2020
<br>
- 2.2. ข้อมูลการเลือกตั้งปี 2016
    - 2.2.1. ข้อมูลผลการเลือกตั้งปี 2016
    - 2.2.2. ข้อมูล Electrical Votes ปี 2016
<br>    
- 2.3. ข้อมูลประชากรระดับ County

In [ ]:
# 2020 US Presidential Election Vote By County
pres_2020 = pd.read_csv('/kaggle/input/us-election-2020/president_county_candidate.csv')

# 2016 US Presidential Election Vote By County
pres_2016 = pd.read_csv('/kaggle/input/2016uspresidentialvotebycounty/pres16results.csv')

# State
state_info = pd.read_csv('/kaggle/input/us-state-county-name-codes/states.csv')
state_info = state_info.rename(columns={'State': 'state', 'Abbreviation': 'abv'})

# Electoral college
electoral_college = pd.read_csv('/kaggle/input/us-electoral-college-votes-per-state-17882020/Electoral_College.csv')

In [ ]:
michigan_pop = pd.read_csv('../input/us-population/cc-michigan-2019.csv')
arizona_pop = pd.read_csv('../input/us-population/cc_arizona_2019.csv')
penn_pop = pd.read_csv('../input/us-population/cc_pennsylvania_2019.csv')
wis_pop = pd.read_csv('../input/us-population/cc_wisconsin_2019.csv')

In [ ]:
print("2020")
print(pres_2020.shape)
display(pres_2020.head())
print("2016")
print(pres_2016.shape)
display(pres_2016.head())

In [ ]:
print("2020")
print(pres_2020.info())
print("============================================")
print("2016")
print(pres_2016.info())

# 3. การสร้างฐานข้อมูลเพื่อการวิเคราะห์ (Data Mart)
- 3.1. การเลือกข้อมูล
        เนื่องจากข้อมูลผลการเลือกตั้งปี 2020 และปี 2016 มีไม่ครบถ้วนทั้ง County และ State
        จึงทำการเลือกมาเฉพาะข้อมูลจาก County และ State ที่มีเหมือนกัน
        > 1808 Counites & 44 States
        
- 3.2. การจัดรูปแบบข้อมูล
        
        
- 3.3. การ clean ข้อมูล
        - 3.3.1. Drop column ที่ไม่ได้ใช้
        - 3.3.2. เปลี่ยนชื่อ column เพื่อให้ชื่อ column consistent ระหว่างข้อมูลเลือกต้ังปี 2020 และ 2016
        - 3.3.3. Drop sample ที่ไม่มีข้อมูล state หรือ county
        - 3.3.4. ใช้ชื่อเต็มของ state แทน abbreviation
        

# Preprocess

In [ ]:
# Drop columns (consistency)
pres_2016.drop(['fips', 'pct_report', 'total_votes', 'pct', 'lead'], axis=1, inplace=True)
pres_2020.drop(['party', 'won'], axis=1, inplace=True)

# Rename columns (consistent with 2020)
pres_2016 = pres_2016.rename(columns={'cand': 'candidate', 'st': 'state', 'votes': 'total_votes'})

# Drop Missing state, county
pres_2016.dropna(inplace=True)

state_map = state_info.reset_index(drop=True).set_index('abv').to_dict()['state']
pres_2016['state'] = pres_2016['state'].map(state_map)

In [ ]:
print("2020")
print(pres_2020.shape)
display(pres_2020.head())
print("2016")
print(pres_2016.shape)
display(pres_2016.head())


# Get intersect counties and states

In [ ]:
# create new df for only DEM and REP
dem_rep_pres_2020 = pres_2020[(pres_2020.candidate == 'Donald Trump') | (pres_2020.candidate == 'Joe Biden')]
dem_rep_pres_2016 = pres_2016[(pres_2016.candidate == 'Donald Trump') | (pres_2016.candidate == 'Hillary Clinton')]

In [ ]:
# Counties 
counties_2020 = dem_rep_pres_2020.county.unique()
counties_2016 = dem_rep_pres_2016.county.unique()

intersect_counties = list(set(counties_2020) & set(counties_2016))
print("Intersect County: ", len(intersect_counties))

# PRES 2020: drop results that not in the intersect counties
to_drop_idx = []
for idx in tqdm(dem_rep_pres_2020.index):
    county = dem_rep_pres_2020.loc[idx, ['county']].values
    if county not in intersect_counties:
        to_drop_idx.append(idx)
        
dem_rep_pres_2020.drop(to_drop_idx, axis=0, inplace=True)

# PRES 2016: drop results that not in the intersect counties
to_drop_idx = []
for idx in tqdm(dem_rep_pres_2016.index):
    county = dem_rep_pres_2016.loc[idx, ['county']].values
    if county not in intersect_counties:
        to_drop_idx.append(idx)
        
dem_rep_pres_2016.drop(to_drop_idx, axis=0, inplace=True)

print("2020's counties , 2016's counties: ", (dem_rep_pres_2020.county.nunique(), dem_rep_pres_2016.county.nunique()))

In [ ]:
# States
states_2020 = dem_rep_pres_2020.state.unique()
states_2016 = dem_rep_pres_2016.state.unique()

intersect_states = list(set(states_2020) & set(states_2016))
print("Intersect State: ", len(intersect_states))

# PRES 2020: drop results that not in the intersect states
to_drop_idx = []
for idx in tqdm(dem_rep_pres_2020.index):
    state = dem_rep_pres_2020.loc[idx, ['state']].values
    if state not in intersect_states:
        to_drop_idx.append(idx)
        
dem_rep_pres_2020.drop(to_drop_idx, axis=0, inplace=True)

# PRES 2016: drop results that not in the intersect states
to_drop_idx = []
for idx in tqdm(dem_rep_pres_2016.index):
    state = dem_rep_pres_2016.loc[idx, ['state']].values
    if state not in intersect_states:
        to_drop_idx.append(idx)
        
dem_rep_pres_2016.drop(to_drop_idx, axis=0, inplace=True)
        
print("2020's states , 2016's states: ", (dem_rep_pres_2020.state.nunique(), dem_rep_pres_2016.state.nunique()))

In [ ]:
print("Original")
print("2020 : {}, 2016: {}".format(pres_2020.shape, pres_2016.shape))
print("Filtered states and counties")
print("2020 : {}, 2016: {}".format(dem_rep_pres_2020.shape, dem_rep_pres_2016.shape))

# 1808 Counties, 44 States

In [ ]:
print("44 states: ")
print(dem_rep_pres_2020.state.unique())
print()
print("except states: ")
except_states = list(set(pres_2020.state.unique()) - set(dem_rep_pres_2020.state.unique())) + list(set(pres_2016.state.unique()) - set(dem_rep_pres_2020.state.unique()))
print(except_states)

# Find who win each county

In [ ]:
winner_2020_county = pd.DataFrame()
winner_2016_county = pd.DataFrame()

def get_winner_county(x):
    diff = abs(x.total_votes.diff().sum())
    summ = x.total_votes.sum()
    win_percent = diff / summ * 100
    result = x.loc[x['total_votes'].idxmax()][['candidate', 'total_votes']]
    result['win_percent'] = win_percent
    
    return result

winner_2020_county = dem_rep_pres_2020.groupby(['state', 'county']).progress_apply(get_winner_county).reset_index()
winner_2016_county = dem_rep_pres_2016.groupby(['state', 'county']).progress_apply(get_winner_county).reset_index()


In [ ]:
# Counties count
print("== #counties winner count: ===")
print("2020")
print(winner_2020_county.candidate.value_counts())
print()
print("2016")
print(winner_2016_county.candidate.value_counts())

> REP win more counites both 2016 and 2020

In [ ]:
# Counties count
print("== #counties total votes winner count: ===")
print("2020")
print(winner_2020_county.groupby('candidate').total_votes.sum().sort_values(ascending=False))
print()
print("2016")
print(winner_2016_county.groupby('candidate').total_votes.sum().sort_values(ascending=False))

> REP lost votes in 2020

# Find who win each State

In [ ]:
winner_2020_state = dem_rep_pres_2020.groupby(['state', 'candidate']).total_votes.sum().sort_values(ascending=False).groupby(level=0).head(1).reset_index()[['state','candidate']]

winner_2016_state = dem_rep_pres_2016.groupby(['state', 'candidate']).total_votes.sum().sort_values(ascending=False).groupby(level=0).head(1).reset_index()[['state','candidate']]


In [ ]:
# Counties count
print("== #states winner count: ===")
print("2020")
print(winner_2020_state.candidate.value_counts())
print()
print("2016")
print(winner_2016_state.candidate.value_counts())

> REP win more states both 2016 and 2020

# Count the Electoral College

In [ ]:
# Get 2020 and 2016 info
ec_2020 = electoral_college.query('Year == 2020').drop('Year', axis=1)
ec_2016 = electoral_college.query('Year == 2016').drop('Year', axis=1)

# Unify state name
ec_2020.loc[ec_2020[ec_2020.State == 'D.C.'].index, 'State'] = 'District of Columbia'
ec_2016.loc[ec_2016[ec_2016.State == 'D.C.'].index, 'State'] = 'District of Columbia'

# Create map
ec_2020_map = ec_2020.reset_index(drop=True).set_index('State').to_dict()['Votes']
ec_2016_map = ec_2016.reset_index(drop=True).set_index('State').to_dict()['Votes']

In [ ]:
winner_2020_state['votes'] = winner_2020_state.state.map(ec_2020_map)
winner_2016_state['votes'] = winner_2016_state.state.map(ec_2016_map)

In [ ]:
# Counties count
print("== #electrical votes winner: ===")
print("2020")
print(winner_2020_state.groupby('candidate').votes.sum().sort_values(ascending=False))
print()
print("2016")
print(winner_2016_state.groupby('candidate').votes.sum().sort_values(ascending=False))


# EDA

In [ ]:
ec_2020[ec_2020.State.isin(except_states)]

Missing Stats Info (doesn't effect the 2020 result)

1. Alaska: (2016, REP) , (2020, REP) 
2. Connecticut : (2016, DEM) , (2020, DEM)
3. Maine: (2016, DEM) , (2020, DEM)
4. Massachusetts: (2016, DEM) , (2020, DEM)
5. New Hampshire: (2016, DEM) , (2020, DEM)
6. Rhode Island: (2016, DEM) , (2020, DEM)
7. Vermonth: (2016, DEM) , (2020, DEM)

In [ ]:
print("From REP to DEM")
changed_state_list = list(set(winner_2016_state[winner_2016_state.candidate == 'Donald Trump'].state.unique()).difference(set(winner_2020_state[winner_2020_state.candidate == 'Donald Trump'].state.unique())))
print("EC votes : ", ec_2020.set_index('State').loc[changed_state_list].Votes.sum())
ec_2020.set_index('State').loc[changed_state_list]

In [ ]:
data = ec_2020.sort_values(by='Votes', ascending=False)
fig = go.Figure(data=[
    go.Bar(name='Total Male 2016', x=data['State'], y=data['Votes'])
])

fig.show()

> REP lose 5 states with high electrical votes to DEM

In [ ]:
print("Compare number of winning counties in these states")
print("2020")
print(winner_2020_county[winner_2020_county.state.isin(changed_state_list)].groupby('candidate').county.count().sort_values(ascending=False))
print()
print("2016")
print(winner_2016_county[winner_2016_county.state.isin(changed_state_list)].groupby('candidate').county.count().sort_values(ascending=False))

In [ ]:
print("Compare votes counties in these states")
print("2020")
print(winner_2020_county[winner_2020_county.state.isin(changed_state_list)].groupby('candidate').total_votes.sum().sort_values(ascending=False))
print()
print("2016")
print(winner_2016_county[winner_2016_county.state.isin(changed_state_list)].groupby('candidate').total_votes.sum().sort_values(ascending=False))

# Which county that made the different

In [ ]:
REP_county_2020 = winner_2020_county[(winner_2020_county.state.isin(changed_state_list)) & (winner_2020_county.candidate == 'Donald Trump')]
DEM_county_2020 = winner_2020_county[(winner_2020_county.state.isin(changed_state_list)) & (winner_2020_county.candidate == 'Joe Biden')]

REP_county_2016 = winner_2016_county[(winner_2016_county.state.isin(changed_state_list)) & (winner_2016_county.candidate == 'Donald Trump')]
DEM_county_2016 = winner_2016_county[(winner_2016_county.state.isin(changed_state_list)) & (winner_2016_county.candidate == 'Hillary Clinton')]

In [ ]:
changed_county_list = list(set(REP_county_2016.county.values) - set(REP_county_2020.county.values))

print("2016")
display(REP_county_2016[REP_county_2016.county.isin(changed_county_list)])
print("2020")
display(DEM_county_2020[DEM_county_2020.county.isin(changed_county_list)])

# Population of each county

In [ ]:
state_pop = {
    'Arizona': arizona_pop,
    'Michigan': michigan_pop,
    'Pennsylvania': penn_pop,
    'Wisconsin': wis_pop,
}

In [ ]:
OTHER_MALE_COLS = ['BA_MALE', 'IA_MALE', 'AA_MALE', 'NA_MALE', 'TOM_MALE']
OTHER_FEMALE_COLS = ['BA_FEMALE', 'IA_FEMALE', 'AA_FEMALE', 'NA_FEMALE', 'TOM_FEMALE']

states = DEM_county_2020[DEM_county_2020.county.isin(changed_county_list)][['state', 'county']]

In [ ]:
def plot_gender(year_2016, year_2020, state_name, county_name):
    age_2016 = year_2016[['AGEGRP', 'TOT_MALE', 'TOT_FEMALE']]
    age_2020 = year_2020[['AGEGRP', 'TOT_MALE', 'TOT_FEMALE']]
    
    pct_fem_2016 = age_2016.apply(lambda x: x[-1]/sum(x[-2:])*100, axis=1)
    pct_fem_2020 = age_2020.apply(lambda x: x[-1]/sum(x[-2:])*100, axis=1)
    
    df_2016 = age_2016.query("AGEGRP > 4").reset_index(drop=True).set_index('AGEGRP')
    df_2016.columns = df_2016.columns.map('{}_2016'.format)
    
    df_2020 = age_2020.query("AGEGRP > 4").reset_index(drop=True).set_index('AGEGRP')
    df_2020.columns = df_2020.columns.map('{}_2020'.format)
    
    df = pd.concat([df_2016, df_2020], axis=1)
    
    fig = go.Figure(data=[
        go.Bar(name='FEMALE 2016', y=df.index, x=df['TOT_FEMALE_2016'], orientation='h', offsetgroup=0, marker_color='rgb(179,205,227)'),
        go.Bar(name='MALE 2016', y=df.index, x=df['TOT_MALE_2016'], orientation='h', offsetgroup=0, marker_color='rgb(55,126,184)'),
        go.Bar(name='FEMALE 2020', y=df.index, x=df['TOT_FEMALE_2020'], orientation='h', offsetgroup=1, marker_color='rgb(251,180,174)'),
        go.Bar(name='MALE 2020', y=df.index, x=df['TOT_MALE_2020'], orientation='h', offsetgroup=1, marker_color='rgb(228,26,28)'),
    ])
    fig.update_layout(xaxis_title="Population",
                      yaxis_title="Age Group",
                      title_text='(GENDER) State: {} --- County: {}'.format(state_name, county_name))
    fig.show();
    
    
def plot_races(year_2016, year_2020, gender, state_name, county_name):
    
    gender_cols = ['WA_MALE'] + OTHER_MALE_COLS
    if gender == 'female':
        gender_cols = ['WA_FEMALE'] + OTHER_FEMALE_COLS
    
    race_2016 = year_2016.reset_index(drop=True).set_index('AGEGRP')[gender_cols]
    race_2016['OTHER'] = year_2016[gender_cols[1:]].sum(axis=1).values
    
    race_2020 = year_2020.reset_index(drop=True).set_index('AGEGRP')[gender_cols]
    race_2020['OTHER'] = year_2020[gender_cols[1:]].sum(axis=1).values
    
    df_2016 = race_2016.loc[5:]
    df_2016.columns = df_2016.columns.map('{}_2016'.format)

    df_2020 = race_2020.loc[5:]
    df_2020.columns = df_2020.columns.map('{}_2020'.format)

    df = pd.concat([df_2016, df_2020], axis=1)

    fig = go.Figure(data=[
        go.Bar(name='WHITE 2016', y=df.index, x=df['WA_{}_2016'.format(gender.upper())],  orientation='h', offsetgroup=0, marker_color='rgb(179,205,227)'),
        go.Bar(name='OTHER 2016', y=df.index, x=df['OTHER_2016'],  orientation='h', offsetgroup=0, marker_color='rgb(55,126,184)'),
        go.Bar(name='WHITE 2020', y=df.index, x=df['WA_{}_2020'.format(gender.upper())],  orientation='h', offsetgroup=1, marker_color='rgb(251,180,174)'),
        go.Bar(name='OTHER 2020', y=df.index, x=df['OTHER_2020'],  orientation='h', offsetgroup=1, marker_color='rgb(228,26,28)'),
    ])
    fig.update_layout(xaxis_title='Population',
                      yaxis_title="Age Group",
                      title_text='(HUMAN RACE) State: {} --- County: {}'.format(state_name, county_name))
    fig.show()

In [ ]:
for row in states.iterrows():
    state_name = row[1].state
    county_name = row[1].county
    cur_pop = state_pop[state_name]
    
    # 9 = 7/1/2016 population estimate 
    # 12 = 7/1/2019 population estimate
    year_2016 = cur_pop[(cur_pop['CTYNAME'] == county_name) & (cur_pop['YEAR'] == 9)]
    year_2020 = cur_pop[(cur_pop['CTYNAME'] == county_name) & (cur_pop['YEAR'] == 12)]

    plot_gender(year_2016, year_2020, state_name, county_name);
    plot_races(year_2016, year_2020, 'male', state_name, county_name);
    plot_races(year_2016, year_2020, 'female', state_name, county_name);

# 4. การหาคำตอบจากข้อมูล

> คำถาม : ทำไม Donald Trump ถึงแพ้การเลือกตั้งปี 2020
<br>
> สมมติฐาน : 
    - 1. Trump เสีย State ซึ่งเคยเป็นฐานเสียงเมื่อการเลือกตั้งปี 2016
      A: Trump เสีย 5 รัฐใหญ่ (Electoral Votes สูง) ['Michigan', 'Georgia', 'Wisconsin', 'Pennsylvania', 'Arizona'] ให้กับ DEM ในปี 2020
    
> คำถาม : อะไรทำให้คะแนน Vote ภายใน State ที่เคยเป็นฐานเสียงของ Trump เมื่อการเลือกตั้งปี 2016 เปลี่ยน
<br>
> สมมติฐาน : 
    - 1. รูปแบบประชากรเปลี่ยนไป 
        - 1.1. New Voters
          A: จำนวน New Voters ภายใน 5 รัฐนั้นเพิ่มขึ้น
        - 1.2. Gender Ratio
          A: Gender Ratio ไม่ได้เปลี่ยนแปลงมากจากปี 2016 ไม่น่าจะส่งผลกับผลการเลือกตั้ง
        - 1.3. Races
          A: จำนวน NON-WHITE ภายใน 5 รัฐนั้นเพิ่มขึ้นจากปี 2016 
    - 2. เพศของ Candidate จากฝั่ง DEM
        - 2.1. Voters หันมาเลือก DEM เพราะ Candidate เป็นผู้ชาย คาดว่า Voters ผู้ชายจะโหวตให้กับ Candidate ผู้ชาย
          A: เนื่องจาก Gender Ratio ไม่ได้เปลี่ยนแปลงมากจากปี 2016 เพศของ Candidate ไม่น่าจะส่งผลต่อการตัดสินใจ


# 5. เกิดคำถามใหม่

-

# 6. การสรุปผล

- 6.1. ข้อมูล States ที่ไม่ได้นำมาวิเคราะห์ประกอบด้วย

    1. Alaska:        (2016, REP) , (2020, REP) 
    2. Connecticut :  (2016, DEM) , (2020, DEM)
    3. Maine:         (2016, DEM) , (2020, DEM)
    4. Massachusetts: (2016, DEM) , (2020, DEM)
    5. New Hampshire: (2016, DEM) , (2020, DEM)
    6. Rhode Island:  (2016, DEM) , (2020, DEM)
    7. Vermonth:      (2016, DEM) , (2020, DEM)
    
แต่เนื่องจาก ผลการเลือกตั้งปี 2016 และ 2020 ของ states เหล่านี้ไม่เปลี่ยนแปลงจึงไม่ส่งผลต่อการวิเคราะห์
<br>
ref: https://thestandard.co/us-election-2020/

- 6.2. การที่จะชนะ State ต้องชนะด้วยคะแนน Votes
        
        **Compare number of winning counties in these states**
        2020
        candidate
        Donald Trump    323
        Joe Biden        73
        Name: county, dtype: int64

        2016
        candidate
        Donald Trump       329
        Hillary Clinton     67
        
        > ถึงแม้ว่า Donald Trump จะชนะในหลาย counties         
        
        **Counties total votes winner count**
        2020
        candidate
        Joe Biden       52661615
        Donald Trump    40297889
        Name: total_votes, dtype: int64

        2016
        candidate
        Donald Trump       37284542.0
        Hillary Clinton    36854558.0
        Name: total_votes, dtype: float64
        
        > แต่จำนวนประชากรภายใน counites เหล่านั้นมีน้อยทำให้คะแนน Votes น้อยและไม่สามารถ ชนะ State ได้
        
        
- 6.3. การชนะการเลือกตั้ง ต้องชนะด้วย Electoral Votes
        **#states winner count**
        2020
        Donald Trump    24
        Joe Biden       20
        Name: candidate, dtype: int64

        2016
        Donald Trump       29
        Hillary Clinton    15
        Name: candidate, dtype: int64
        
        > ถึงแม้ว่า Donald Trump จะชนะในหลาย states
        
        **Compare votes counties in these states**
        2020
        candidate
        Joe Biden       8206029
        Donald Trump    6665976
        Name: total_votes, dtype: int64

        2016
        candidate
        Donald Trump       6467614.0
        Hillary Clinton    5458081.0
        Name: total_votes, dtype: float64
        
        > แต่จำนวน Electoral Votes ภายใน state เหล่านั้นมีน้อยทำให้คะแนน Votes น้อยและไม่สามารถ ชนะการเลือกตั้งได้ ได้
        
- 6.4. Trump เสีย 5 รัฐใหญ่ (Electoral Votes สูง) [Michigan, Georgia, Wisconsin, Pennsylvania, Arizona] ให้กับ DEM ในปี 2020
<br>Biden ชนะ 14 รัฐใหญ่จาก 20
                      State	       Votes
       (Biden)  0	 California      55.0
       (Trump)  1	 Texas           38.0
       (Trump)  2	 Florida	     29.0
       (Biden)  3	 New York	     29.0
       (Biden)  4	 Illinois	     20.0
        >>>     5	 Pennsylvania	 20.0 
       (Trump)  6	 Ohio	         18.0
        >>>     7	 Georgia	     16.0
        >>>     8	 Michigan	     16.0
       (Trump)  9	 North Carolina	 15.0
       (Biden)  10	New Jersey	     14.0
       (Biden)  11	Virginia	     13.0
       (Biden)  12	Washington	     12.0
       (Trump)  13	Tennessee	     11.0
       (Trump)  14	Indiana	         11.0
        >>>     15	Arizona	         11.0
       (Biden)  16	Massachusetts	 11.0
       (Biden)  17	Minnesota	     10.0
       (Biden)  18	Maryland	     10.0
       (Trump)  19	Missouri	     10.0
        >>>     20	Wisconsin	     10.0
        



- 6.5. สิ่งที่ทำให้คะแนน Vote ภายใน State ที่เคยเป็นฐานเสียงของ Trump เมื่อการเลือกตั้งปี 2016 เปลี่ยน
        ข้อสมมติฐาน (เนื่องจากไม่มีข้อมูลมายืนยัน)
        1. New Voter
            > จำนวน New Voter เพิ่มมากขึ้น มีความเป็นไปได้ว่าพวกเค้าโหวตเลือก DEM
        2. Human Race
            > จำนวน NON-WHITE เพิ่มมากขึ้น มีความเป็นไปได้ว่าพวกเค้าโหวตเลือก DEM เนื่องจากนโยบายของ DEM ค่อนข้างจะยืดหยุ่นให้กับคนเหล่านี้
        3. อาจมี Voter บางส่วนที่ไม่ชอบการทำงานของ Trump และหันไปเลือก DEM

Ref:
1. Population dataset
population dataset: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-detail.html
<br>
file layout: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2019/cc-est2019-alldata.pdf